In [71]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing import image

In [72]:
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.layers import Dense
from keras import Sequential

In [73]:
df=pd.read_csv('_classes.csv')

In [74]:
df.head()

,filename,with_mask,without_mask
0,26-with-mask_jpg.rf.fb2a4fd14461ba4e7d981f51d9...,1,0
1,66_jpg.rf.fb48333563aba0cd06afba343f625d98.jpg,0,1
2,153-with-mask_jpg.rf.fb584ce29ce6d9a3123a04d80...,1,0
3,29-with-mask_jpg.rf.fb9ecac8d3dc335a43b79a9438...,1,0
4,augmented_image_315_jpg.rf.fba9c25c6675c5a039f...,1,0


In [75]:
os.chdir('train')

data=[]
for i,j in zip(df['filename'], df[' with_mask']):
  img = cv2.imread(i)
  img=cv2.resize(img,(224,224))
  data.append([img,j])

In [76]:
len(data)

1279

In [77]:
X=[]
y=[]

for features, labels in data:
  X.append(features)
  y.append(labels)

In [78]:
X=np.array(X)
y=np.array(y)

In [79]:
print(X.shape)
print(y.shape)

(1279, 224, 224, 3)
(1279,)


In [80]:
X=X/255 #scaling

In [81]:
x_train,x_test,y_train, y_test=train_test_split(X,y, test_size=0.2,random_state=1)

In [82]:
vgg=VGG16()

In [83]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [84]:
model=Sequential()

In [85]:
for layers in vgg.layers[:-1]:
  model.add(layers)

In [86]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [87]:
for layer in model.layers:
  layer.trainable=False

In [88]:
model.add(Dense(1,activation ='sigmoid'))

In [89]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [90]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [91]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
32/32 [==============================] - 180s 6s/step - loss: 0.5895 - accuracy: 0.6735 - val_loss: 0.5096 - val_accuracy: 0.7188
Epoch 2/5
32/32 [==============================] - 173s 5s/step - loss: 0.3344 - accuracy: 0.9140 - val_loss: 0.2994 - val_accuracy: 0.9102
Epoch 3/5
32/32 [==============================] - 168s 5s/step - loss: 0.2448 - accuracy: 0.9355 - val_loss: 0.2491 - val_accuracy: 0.9258
Epoch 4/5
32/32 [==============================] - 169s 5s/step - loss: 0.2055 - accuracy: 0.9433 - val_loss: 0.2136 - val_accuracy: 0.9336
Epoch 5/5
32/32 [==============================] - 167s 5s/step - loss: 0.1777 - accuracy: 0.9472 - val_loss: 0.1897 - val_accuracy: 0.9375


In [101]:
# Create a video capture object
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Capture frame-by-frame
while True:
    ret, frame = cap.read()
    img=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
    #rint(y_pred)
    if y_pred<0.95:
        draw_label(frame, 'No Mask',(30,30),(0,0,255))
    else:
        draw_label(frame, 'Mask',(30,30),(0,255,0))
    
    # Display the frame
    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF==ord('x'):
        break

# Release the capture object
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 161ms/step


1/1 [==============================] - 0s 204ms/step


In [93]:
def detect_face_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    return y_pred
    

In [94]:
# Get the current working directory
current_directory = os.getcwd()

# Get the parent directory
parent_directory = os.path.dirname(current_directory)

# Change the working directory to the parent directory
os.chdir(parent_directory)

# Print the new working directory
print(os.getcwd())



C:\Users\815ga\CNN facemask detector


In [95]:
%cd CNN facemask detector

[WinError 2] The system cannot find the file specified: 'CNN facemask detector'
C:\Users\815ga\CNN facemask detector


In [96]:
sample1=cv2.imread('14.jpg')
sample1=cv2.resize(sample1,(224,224))

In [97]:
detect_face_mask(sample1)

1/1 [==============================] - 0s 235ms/step


array([[0.17696276]], dtype=float32)

In [98]:
def draw_label(img,text,pos,bg_color):
    
    text_size= cv2.getTextSize(text,cv2.FONT_HERSHEY_TRIPLEX,1,cv2.FILLED)
    
    end_x=pos[0]+ text_size[0][0] + 2
    end_y= pos[1]+ text_size[0][1] -2
    
    cv2.rectangle(img, pos, (end_x,end_y), bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_TRIPLEX,1,(0,0,0),1,cv2.LINE_AA)
    
